In [61]:
%matplotlib inline

from modsim import *
import numpy as np

In [62]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton

In [63]:
condition = Condition(height = 381 * m,
                      v_init = 0 * m / s,
                      g = 9.8 * m/s**2,
                      mass = 75 * kg,
                      rho = 1.2 * kg/m**3,
                      v_term = 60 * m / s,
                      duration = 40 * s)

In [64]:
def final_state(results):
    """Returns the final position and velocity, with units.
    
    results: TimeFrame with y and v.
    
    returns: y, v at t_end
    """
    t_end = results.index[-1]
    y, v = results.loc[t_end]
    return y*m, v*m/s

In [65]:
def make_system(condition, area):
    """Makes a System object for the given conditions.
    
    condition: Condition with height, g, mass, diameter, 
               rho, v_term, and duration
    
    returns: System with init, g, mass, rho, C_d, area, and ts
    """
    unpack(condition)
    
    init = State(y=height, v=v_init)
    C_d = 2 * mass * g / (rho * (area*m**2) * v_term**2)
    ts = linspace(0, duration, 101)
    
    return System(init=init, g=g, mass=mass, rho=rho,
                  C_d=C_d, area=area*m**2,
                  y_attach=height, ts=ts)

In [66]:
def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing g, rho,
            C_d, area, and mass
    
    returns: derivatives of y and v
    """
    y, v = state
    unpack(system)
    
    f_drag = rho * v**2 * C_d * area / 2
    a_drag = f_drag / mass
    
    dydt = v
    dvdt = -g + a_drag
    if a_drag <= g:
        return dydt, dvdt

In [67]:
def clusterfuck(areas):
    mylist = []
    for area in areas:
        system= make_system(condition, area*m**2)
        slope_func(system.init, 0, system)
        run_odeint(system, slope_func)
        T = interp_inverse(system.results.y,kind='cubic')
        ground=T(0)
        condition.set(duration=ground)
        system = make_system(condition, area*m**2)
        run_odeint(system, slope_func)
        y_final, v_final = final_state(system.results)
        mylist.append(v_final)
    return mylist

In [76]:
areas = linspace(1,300,300, dtype = int)
po =clusterfuck(area)
myarray = np.asarray(int(po))
plot(areas,myarray)

TypeError: 'numpy.float64' object is not iterable

In [87]:
slope_func(system.init, 0, system)

(<Quantity(0.0, 'meter / second')>, <Quantity(-9.8, 'meter / second ** 2')>)

In [88]:
run_odeint(system, slope_func)

In [89]:
system.results

,y,v
0.0,381.000000,0.000000
0.4,380.216561,-3.914432
0.8,377.872889,-7.795684
1.2,373.988725,-11.611688
1.6,368.596240,-15.332534
2.0,361.738985,-18.931348
2.4,353.470537,-22.384988
2.8,343.852899,-25.674499
3.2,332.954781,-28.785332
3.6,320.849829,-31.707340
